In [1]:
from pathlib import Path

import pandas as pd

from src.calculate_pairs import generate_complete_sets
from src.csv_writer import write_to_csv
from src.image_maker import draw_splines_on_circular_numbers

In [2]:
n = 42

In [ ]:
all_sets = generate_complete_sets(n_numbers=n)
write_to_csv(all_sets, output_csv=f"data/complete_sets_n{n}.csv")

In [ ]:
from src.sort_overlapping_lists import sort_lists_smart

ordered = sort_lists_smart(all_sets)

write_to_csv(ordered, output_csv=f"data/ordered_sets_n{n}.csv")

In [4]:
max_images = 0

# Read from CSV
df = pd.read_csv("data/ordered_sets_n42.csv")

# Clear images directory
images_dir = Path("images")
if images_dir.exists() and images_dir.is_dir():
    for file in images_dir.iterdir():
        if file.is_file():
            file.unlink()


# Iterate through the rows to process one row at a time
for index, row in df.iterrows():
    # Extract pairs from the row (assuming pairs are in the format "a-b").
    # Skip the first column which has Pair No.
    pairs = []
    for col in df.columns[1:]:
        pair_str = row[col]
        if pd.isna(pair_str):
            continue  # Skip if the cell is empty
        a, b = map(int, pair_str.split("-"))
        pairs.append((a, b))

    draw_splines_on_circular_numbers(n, pairs, f"images/n{n}_{index:06}.png")

    if max_images and index > max_images:
        break

Image with splines saved to images/n42_000000.png
Image with splines saved to images/n42_000001.png
Image with splines saved to images/n42_000002.png
Image with splines saved to images/n42_000003.png
Image with splines saved to images/n42_000004.png
Image with splines saved to images/n42_000005.png
Image with splines saved to images/n42_000006.png
Image with splines saved to images/n42_000007.png
Image with splines saved to images/n42_000008.png
Image with splines saved to images/n42_000009.png
Image with splines saved to images/n42_000010.png
Image with splines saved to images/n42_000011.png
Image with splines saved to images/n42_000012.png
Image with splines saved to images/n42_000013.png
Image with splines saved to images/n42_000014.png
Image with splines saved to images/n42_000015.png
Image with splines saved to images/n42_000016.png
Image with splines saved to images/n42_000017.png
Image with splines saved to images/n42_000018.png
Image with splines saved to images/n42_000019.png


In [5]:
from src.video_maker import create_video_from_images

# # # Basic usage
create_video_from_images(
    image_folder="images/",
    output_video=f"video/n{n}.mp4",
    fps=30,
    frames_per_image=5,
)


# Basic usage - smooth transitions between images
# create_video_with_transitions(
#     image_folder="images/",
#     output_video=f"video/n{n}.mp4",
#     fps=30,
#     frames_per_image=30,
#     transition_frames=15,  # 0.5 second crossfade transitions
# )

Found 2603 images
Video settings: 30 FPS, 5 frames per image
Each image will be shown for 0.17 seconds
Total video duration: 433.83 seconds (7.23 minutes)
Video dimensions: 1785x1785
Processing image 1/2603: n42_000000.png
Processing image 2/2603: n42_000001.png
Processing image 3/2603: n42_000002.png
Processing image 4/2603: n42_000003.png
Processing image 5/2603: n42_000004.png
Processing image 6/2603: n42_000005.png
Processing image 7/2603: n42_000006.png
Processing image 8/2603: n42_000007.png
Processing image 9/2603: n42_000008.png
Processing image 10/2603: n42_000009.png
Processing image 11/2603: n42_000010.png
Processing image 12/2603: n42_000011.png
Processing image 13/2603: n42_000012.png
Processing image 14/2603: n42_000013.png
Processing image 15/2603: n42_000014.png
Processing image 16/2603: n42_000015.png
Processing image 17/2603: n42_000016.png
Processing image 18/2603: n42_000017.png
Processing image 19/2603: n42_000018.png
Processing image 20/2603: n42_000019.png
Proces

'video/n42.mp4'

## Converting from raw CSV for n=60

In [ ]:
def parse_pair(pair_str: str) -> tuple[int, int]:
    a, b = map(int, pair_str.split("-"))
    return a, b

In [ ]:
# Load only first 100 rows of CSV
df = pd.read_csv("data/raw_sets_n60.csv")

# Create a list of lists where each row is a list of pairs [a-b]
all_pairs = []
for index, row in df.iterrows():
    pairs = []
    for col in df.columns:  # Skip the first column (Pair No.)
        pair_str = row[col]
        if pd.isna(pair_str):
            continue  # Skip empty cells
        pairs.append(parse_pair(pair_str))
    all_pairs.append(pairs)

all_pairs[0]

In [ ]:
len(all_pairs)

In [ ]:
write_to_csv(all_pairs, output_csv="data/test_sets_n60.csv")